# Descripción del Taller 4 (parte 1) de clase:
---
Trabajar con **servidor** MongoDB
1. Instalar librerias necesarias
2. configuraciones especiales para crear un servidor MongoDB
3. crer un BD
4. Crear colecciones
5. Descargar Dataset's (varios) en un .zip y cargarlos en colecciones de MongoDB

# 0.instalar librerias especiales

In [ ]:
!apt update
!apt install -y python3-pip
!pip3 install pymongo        #libreria especial ()

## 0.1.instalar la versión  mongoDB Ubuntu 22.04

In [ ]:
!sudo apt-get install -y gnupg curl
!curl -fsSL https://pgp.mongodb.com/server-6.0.asc | sudo gpg -o /usr/share/keyrings/mongodb-server-6.0.gpg --dearmor
!echo "deb [ arch=amd64,arm64 signed-by=/usr/share/keyrings/mongodb-server-6.0.gpg ] https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/6.0 multiverse" | sudo tee /etc/apt/sources.list.d/mongodb-org-6.0.list

In [ ]:
!sudo apt-get update
!sudo apt-get install -y mongodb-org

# 1.instanciar las librerias

In [4]:
# habilitamos drive de google desde colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os
db_path = '/content/drive/MyDrive/UniversidadCentral/Maestría_en_Analítica_de_Datos/Bigdata/Ejercicios_de_clase/DataBase/MongoDB/'
os.makedirs(db_path, exist_ok=True)
!sudo chmod 777 $db_path  #permisos especiales de escritura sobre esa carpeta

In [6]:
#modifico las variables de entorno del SO para que reconozca el servidor
!sudo systemctl stop mongod
!sudo mongod --dbpath $db_path --fork --logpath /var/log/mongodb/mongod.log

System has not been booted with systemd as init system (PID 1). Can't operate.
Failed to connect to bus: Host is down
about to fork child process, waiting until server is ready for connections.
forked process: 3831
child process started successfully, parent exiting


# 2.Inicializar el servidor de MOngoDB

In [50]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)

###2.1 crear BD y coleccion

In [53]:
db = client['estudiantes_central']

#-------------coleciones (tablas)---------

colle_profesores  =db['profesores']
colle_cursos      =db['cursos']

##2.2 funciones de DML
---
(select, insert, update, delete)

In [21]:
def consultar_documentos(coleccion, filtro=None):
  """imprimir listado de documentos"""
  try:
    if filtro:
      documents = coleccion.find(filtro)
    else:
      documents = coleccion.find()
    for document in documents:
      print(document)
  except Exception as e:
    print(f"Error al consultar documentos: {e}")

def consultar_total_documentos(coleccion):
  """imprimir total de documentos"""
  try:
    total_documents = coleccion.count_documents({})
    print(f"Total de documentos en la colección: {total_documents}")
  except Exception as e:
    print(f"Error al consultar total de documentos: {e}")

def crear_documento(coleccion, datos):
  """crear un documento"""
  try:
    result = coleccion.insert_one(datos)
    print(f"Documento creado con ID: {result.inserted_id}")
  except Exception as e:
    print(f"Error al crear documento: {e}")

def actualizar_documento(coleccion, filtro, nuevos_valores):
  """actualizar un documento"""
  try:
    result = coleccion.update_one(filtro, {"$set": nuevos_valores})
    if result.modified_count > 0:
      print("Documento actualizado")
    else:
      print("No se encontraron documentos para actualizar")
  except Exception as e:
    print(f"Error al actualizar documento: {e}")

In [55]:
crear_documento(colle_profesores,{"nombre":"pepito Perez","Profesion":"Abogado","especialidad":"Derecho laboral"})
consultar_documentos(colle_profesores)
consultar_total_documentos(colle_profesores)

Documento creado con ID: 67ca3949361b752a5309e8d2
{'_id': ObjectId('67ca3721361b752a5309e8cc'), 'nombre': 'pepito Perez', 'Profesion': 'Abogado', 'especialidad': 'Derecho laboral'}
{'_id': ObjectId('67ca372e361b752a5309e8cd'), 'nombre': 'Andrea Gomez', 'Profesion': 'Ingeniera'}
{'_id': ObjectId('67ca3949361b752a5309e8d2'), 'nombre': 'pepito Perez', 'Profesion': 'Abogado', 'especialidad': 'Derecho laboral'}
Total de documentos en la colección: 3


In [35]:
actualizar_documento(colle_profesores,{"nombre":"LuisFdo"},{"nombre":"Luis Fernando Castellanos"})
consultar_documentos(colle_profesores)

No se encontraron documentos para actualizar
{'_id': ObjectId('67c9f21e8dcc3c831bbc2254'), 'nombre': 'Luis Fernando Castellanos', 'especialidad': 'BigData'}
{'_id': ObjectId('67ca3721361b752a5309e8cc'), 'nombre': 'pepito Perez', 'Profesion': 'Abogado', 'especialidad': 'Derecho laboral'}
{'_id': ObjectId('67ca372e361b752a5309e8cd'), 'nombre': 'Andrea Gomez', 'Profesion': 'Ingeniera'}


In [34]:
profe_id      = colle_profesores.insert_one({"nombre":"Andrea Gomez","Profesion":"Ingeniera"}).inserted_id
crear_documento(colle_cursos,{"nombre":"BigData",
                         "codigo":"UcentraLBig2025",
                         "horarios":[
                             {"dia":"Jueves","hora":"18:00"},
                             {"dia":"Jueves","hora":"20:00"}
                         ],
                         "profesor":profe_id})

Documento creado con ID: 67ca372e361b752a5309e8ce


##2.3 Consultas MongoDB

In [69]:
#listar las bases de datos existentes y colecciones (tablas)
database_names = client.list_database_names()
for db_name in database_names:
  print("Base de datos: ",db_name)
  db = client[db_name]
  collection_names = db.list_collection_names()
  for collection_name in collection_names:
    print("  Colección: ",collection_name)

Base de datos:  admin
  Colección:  system.version
Base de datos:  config
  Colección:  system.sessions
Base de datos:  estudiantes_central
  Colección:  ingenieria
  Colección:  profesores
  Colección:  diccionario
Base de datos:  local
  Colección:  startup_log


##2.4 Eliminar dentro de mongoDB

### 2.4.1 eliminar una BD y sus colecciones

In [40]:
#para cerrar una instancia
client.close()

In [26]:
from pymongo import MongoClient
clientPaBorrar = MongoClient('localhost', 27017)
db_name="Taller4"
try:
  db=clientPaBorrar[db_name]
  #-----borrado de coleciones---------
  collection_names = db.list_collection_names()
  for collection_name in collection_names:
    db[collection_name].drop()
    print(f"Colección '{collection_name}' eliminada exitosamente.")
  #-----borrado de la bd----------
  clientPaBorrar.drop_database(db_name)
  print(f"Base de datos '{db_name}' eliminada exitosamente.")
except Exception as e:
  print(f"Error al eliminar la base de datos: {e}")
clientPaBorrar.close()

Colección 'cursos' eliminada exitosamente.
Colección 'profesores' eliminada exitosamente.
Base de datos 'Taller4' eliminada exitosamente.


### 2.4.2 eliminar una colección

In [49]:
# prompt: borrar de la base de datos estudiantes_central la colecion  cursos de mongodb
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client['estudiantes_central']

try:
  db['cursos'].drop()
  print("Colección 'cursos' eliminada exitosamente.")
except Exception as e:
  print(f"Error al eliminar la colección 'cursos': {e}")

client.close()


Colección 'cursos' eliminada exitosamente.


### 2.4.3 borrar un registro

In [56]:
criterio_borrado = {"nombre":"pepito Perez"}
result= colle_profesores.delete_one(criterio_borrado)
if result.deleted_count > 0:
  print("Documento borrado")
else:
  print("No se encontraron documentos para borrar")
consultar_documentos(colle_profesores)

Documento borrado
{'_id': ObjectId('67ca372e361b752a5309e8cd'), 'nombre': 'Andrea Gomez', 'Profesion': 'Ingeniera'}
{'_id': ObjectId('67ca3949361b752a5309e8d2'), 'nombre': 'pepito Perez', 'Profesion': 'Abogado', 'especialidad': 'Derecho laboral'}


#3 ETL de un CSV publico (zip)
---
https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip


## 3.1 descargar y descomprimir en carpeta de google drive

In [60]:
def descargar_y_descomprimir_zip(url, destino):
  import pandas as pd
  import requests
  import zipfile
  import io
  import os
  #definiendo la URL donde esta el ZIP
  #url_zip ="https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip"
  #path_dir="/content/drive/MyDrive/UniversidadCentral/Maestría_en_Analítica_de_Datos/Bigdata/Ejercicios_de_clase/DataBase/DataSet-Estudiantes/"
  #crear directorio en caso de no existir
  os.makedirs(destino, exist_ok=True)
  response = requests.get(url)
  zip_file = zipfile.ZipFile(io.BytesIO(response.content))
  #zip_file.extractall(path_dir)   #extraer todo sin importar
  #----extraer los archivo por extensión---
  for file_name in zip_file.namelist():
    if file_name.endswith(('.csv','.txt')):     #extraer unicamente los CSV y txt
      zip_file.extract(file_name, path=destino)


In [61]:
url_zip ="https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip"
path_dir="/content/drive/MyDrive/UniversidadCentral/Maestría_en_Analítica_de_Datos/Bigdata/Ejercicios_de_clase/DataBase/DataSet-Estudiantes/"
descargar_y_descomprimir_zip(url_zip,path_dir)

## 3.2 cargar data del diccionario en una colección

In [68]:
import re

def procesar_data_diccionario_separado_por_espacio(data_diccionario):
  lineas = data_diccionario.strip().split('\n')
  #----inicializar la lista
  documentos=[]
  for linea in lineas:
    #----separar clave----
    match = re.match('^\d+\s+(.+?)\s*-\s*(.+)$',linea)
    if match:
      clave = match.group(1)
      descripcion = match.group(2)
      documentos.append({"clave":clave,"descripcion":descripcion})
  return documentos


In [65]:
#----------cargar el archivo student.txt
with open(path_dir+"student.txt","r") as file:
  data_diccionario=file.read()
documentos= procesar_data_diccionario_separado_por_espacio(data_diccionario)

#crear los documentos en una nueva coleción
db                = client['estudiantes_central']
colle_diccionario = db['diccionario']
if documentos:
  #for documento in documentos:
    #colle_diccionario.insert_one(documento)
  colle_diccionario.insert_many(documentos)
  print("Documentos ",len(documentos)," insertados exitosamente.")
else:
  print("No se encontraron documentos para insertar.")

Documentos  33  insertados exitosamente.


In [ ]:
consultar_total_documentos(colle_diccionario)
consultar_documentos(colle_diccionario)

##3.3 Cargar los Dataset's a colecciones

### 3.3.1 cargar en DataFrame's los dataSet's

In [70]:
import pandas as pd
df_students_mat= pd.read_csv(path_dir+"student-mat.csv",sep=";")
df_students_por= pd.read_csv(path_dir+"student-por.csv",sep=";")

In [71]:
print("primeras 5 filas del dataset student-mat")
df_students_mat.head()

primeras 5 filas del dataset student-mat


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


In [73]:
students_mat_list = df_students_mat.to_dict(orient='records')
students_pro_list = df_students_por.to_dict(orient='records')

db                = client['estudiantes_central']
colle_studiantes  = db['estudiantes']

#____cargar los listados de estudiantes de matematicas
if students_mat_list:
  colle_studiantes.insert_many(students_mat_list)
  print("Documentos ",len(students_mat_list)," insertados exitosamente.")
else:
  print("No se encontraron documentos para insertar.")

#___cargar listados de estudiantes de portugues
if students_pro_list:
  colle_studiantes.insert_many(students_pro_list)
  print("Documentos ",len(students_pro_list)," insertados exitosamente.")
else:
  print("No se encontraron documentos para insertar.")

Documentos  395  insertados exitosamente.
Documentos  649  insertados exitosamente.


In [74]:
consultar_total_documentos(colle_studiantes)
consultar_documentos(colle_studiantes)

Total de documentos en la colección: 1044
{'_id': ObjectId('67ca4dcb361b752a5309e8f4'), 'school': 'GP', 'sex': 'F', 'age': 18, 'address': 'U', 'famsize': 'GT3', 'Pstatus': 'A', 'Medu': 4, 'Fedu': 4, 'Mjob': 'at_home', 'Fjob': 'teacher', 'reason': 'course', 'guardian': 'mother', 'traveltime': 2, 'studytime': 2, 'failures': 0, 'schoolsup': 'yes', 'famsup': 'no', 'paid': 'no', 'activities': 'no', 'nursery': 'yes', 'higher': 'yes', 'internet': 'no', 'romantic': 'no', 'famrel': 4, 'freetime': 3, 'goout': 4, 'Dalc': 1, 'Walc': 1, 'health': 3, 'absences': 6, 'G1': 5, 'G2': 6, 'G3': 6}
{'_id': ObjectId('67ca4dcb361b752a5309e8f5'), 'school': 'GP', 'sex': 'F', 'age': 17, 'address': 'U', 'famsize': 'GT3', 'Pstatus': 'T', 'Medu': 1, 'Fedu': 1, 'Mjob': 'at_home', 'Fjob': 'other', 'reason': 'course', 'guardian': 'father', 'traveltime': 1, 'studytime': 2, 'failures': 0, 'schoolsup': 'no', 'famsup': 'yes', 'paid': 'no', 'activities': 'no', 'nursery': 'no', 'higher': 'yes', 'internet': 'yes', 'romantic